In [126]:
import fiftyone as fo
from fiftyone import ViewField as F
import glob

### Loading Dataset

In [4]:
ds = fo.load_dataset('D2S')

In [5]:
ds.head()

[<Sample: {
     'id': '5fc0e41edb8a866086cd1454',
     'media_type': 'image',
     'filepath': 'C:\\Users\\1\\Downloads\\fo\\D2S_009300.jpg',
     'tags': BaseList([]),
     'metadata': None,
     'ground_truth': <Detections: {
 
         'detections': BaseList([
 
             <Detection: {
 
 
                 'id': '5fc0e41edb8a866086cd1453',
 
 
                 '_id': '5fc0e41edb8a866086cd1453',
 
 
                 'attributes': BaseDict({}),
 
 
                 'label': 'banana_single',
 
 
                 'bounding_box': BaseList([
 
 
                     0.4166666666666667,
 
 
                     0.2875,
 
 
                     0.2140625,
 
 
                     0.38958333333333334,
 
 
                 ]),
 
 
                 'mask': None,
 
 
                 'confidence': None,
 
 
                 'index': None,
 
 
                 '_cls': 'Detection',
 
 
             }>,
 
         ]),
 
         '_cls': 'Detections',
 
     }>,
     'prediction': None,
 }>,
 <

### Adding train samples to the dataset

In [81]:
# PATHs
SAMPLES_JSON_PATH = r'C:\Users\1\Downloads\test_banana_can.json'
SAMPLES_IMAGES_PATH = r'C:\Users\1\Downloads\test_banana_can' 

PREDICTIONS_PATH = r'C:\Users\1\Downloads\predicted_banana_can_new.json'
SAMPLES_JSON_PATH, SAMPLES_IMAGES_PATH, PREDICTIONS_PATH

('C:\\Users\\1\\Downloads\\test_banana_can.json',
 'C:\\Users\\1\\Downloads\\test_banana_can',
 'C:\\Users\\1\\Downloads\\predicted_banana_can_new.json')

In [22]:
# Load JSON
import json
with open(SAMPLES_JSON_PATH) as json_file: 
    samples_anns = json.load(json_file)

In [24]:
with open(PREDICTIONS_PATH) as json_file:
    pred_anns = json.load(json_file)

### Adding ground truth & predictions:

In [84]:
for filepath in glob.glob(SAMPLES_IMAGES_PATH + '\\*'):
    sample = fo.Sample(filepath=filepath)
    
    for im in samples_anns['images']:
        if im['file_name'] == filepath[-14:]:
            image_id = im['id']
            h = float(im['height'])
            w = float(im['width'])
            break
    
    truth_detections = []
    for ann in samples_anns['annotations']:
        if ann['image_id'] == image_id:
            cat_id = ann['category_id']
            for cat in test_anns['categories']:
                if cat['id'] == cat_id:
                    label = cat['name']

            bounding_box = ann['bbox']
            bounding_box = [float(cor) for cor in bounding_box]
            bounding_box[0], bounding_box[2] = bounding_box[0] / w, bounding_box[2] / w
            bounding_box[1], bounding_box[3] = bounding_box[1] / h, bounding_box[3] / h
            
            truth_detections.append(fo.Detection(label=label, bounding_box=bounding_box))

    pred_detections = []
    for ann in pred_anns['annotations']:
        if ann['image_id'] == image_id:
            cat_id = ann['category_id']
            
            for cat in pred_anns['categories']:
                if cat['id'] == cat_id:
                    label = cat['name']
                    
            bounding_box = ann['bbox']
            bounding_box = [float(cor) for cor in bounding_box]
            bounding_box[0], bounding_box[2] = bounding_box[0] / w, bounding_box[2] / w
            bounding_box[1], bounding_box[3] = bounding_box[1] / h, bounding_box[3] / h
            
            confidence = 0.5  # put the confidence if it exists
            pred_detections.append(fo.Detection(label=label, bounding_box=bounding_box, confidence=confidence))
   
    sample["ground_truth"] = fo.Detections(detections=truth_detections)
    sample["prediction"] = fo.Detections(detections=pred_detections)
    sample["tags"] = ["test"]
    ds.add_sample(sample)

## Creating Dataset (without predictions)

In [5]:
ds = fo.Dataset(name='D2S')

In [12]:
IMAGES_PATH = 'C:/Users/1/Downloads/fo/' # Path to the folder with images
ANNOTATION_PATH = 'C:/Users/1/Downloads/fo_test.json' # Path to the annotations

In [13]:
import json
with open(ANNOTATION_PATH) as json_file: 
    anns = json.load(json_file)
len(anns['images'])

120

In [83]:
for filepath in glob.glob(IMAGES_PATH + '*'):
    sample = fo.Sample(filepath=filepath)
    
    for im in anns['images']:
        if im['file_name'] == filepath[-14:]:
            image_id = im['id']
            h = float(im['height'])
            w = float(im['width'])
            break
    
    detections = []
    for ann in anns['annotations']:
        if ann['image_id'] == image_id:
            cat_id = ann['category_id']
            label = anns['categories'][cat_id]['name']

            bounding_box = ann['bbox']
            bounding_box = [float(cor) for cor in bounding_box]
            bounding_box[0], bounding_box[2] = bounding_box[0] / w, bounding_box[2] / w
            bounding_box[1], bounding_box[3] = bounding_box[1] / h, bounding_box[3] / h
            
            detections.append(fo.Detection(label=label, bounding_box=bounding_box))

    sample['ground_truth'] = fo.Detections(detections=detections)
    sample['tags'] = ['train']
    ds.add_sample(sample)

### Save your dataset

In [33]:
ds.persistent = True
ds.save()

### Getting list of datasets

In [23]:
fo.list_datasets()

['D2S']

### Delete dataset

In [ ]:
fo.delete_dataset('D2S') 

### Clear dataset

In [95]:
ds.clear()

Name:           D2S
Media type:     image
Num samples:    140
Persistent:     True
Info:           {}
Tags:           ['test', 'train']
Sample fields:
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    prediction:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

### Saving samples to JSON (search by TAG)

In [87]:
export_dir = r'C:\Users\1\Downloads\export_dir'
dataset_type = fo.types.COCODetectionDataset 
ds.match_tag('test').export(export_dir=export_dir, dataset_type=dataset_type)

Directory 'C:\Users\1\Downloads\export_dir' already exists; export will be merged with existing files
 100% |█████████| 20/20 [1.1s elapsed, 0s remaining, 18.2 samples/s]         


### Saving samples to TFRecord (search by TAG)

In [88]:
export_dir = r'C:\Users\1\Downloads\export_dir'

dataset_type = fo.types.TFObjectDetectionDataset
ds.match_tag('test').export(
    export_dir=export_dir, dataset_type=dataset_type)

Directory 'C:\Users\1\Downloads\export_dir' already exists; export will be merged with existing files
 100% |█████████| 20/20 [544.2ms elapsed, 0s remaining, 36.7 samples/s]      


### Saving samples to JSON (search by CLASS)

In [130]:
CLASSES = ['banana']
samples = ds.filter_labels("prediction", F("label").is_in(CLASSES), only_matches=True)
samples

Dataset:        D2S
Media type:     image
Num samples:    13
Tags:           ['test']
Sample fields:
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    prediction:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
Pipeline stages:
    1. FilterLabels(field='prediction', filter={'$in': ['$$this.label', [...]]}, only_matches=True)

In [128]:
export_dir = r'C:\Users\1\Downloads\export_dir'
dataset_type = fo.types.COCODetectionDataset 
samples.export(export_dir=export_dir, dataset_type=dataset_type)

Directory 'C:\Users\1\Downloads\export_dir' already exists; export will be merged with existing files
 100% |█████████| 13/13 [730.2ms elapsed, 0s remaining, 19.2 samples/s]      


### Saving samples to TFRecord (search by CLASS)

In [132]:
CLASSES = ['can']
samples = ds.filter_labels("ground_truth", F("label").is_in(CLASSES), only_matches=True)
samples

Dataset:        D2S
Media type:     image
Num samples:    11
Tags:           ['test']
Sample fields:
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    prediction:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
Pipeline stages:
    1. FilterLabels(field='ground_truth', filter={'$in': ['$$this.label', [...]]}, only_matches=True)

In [129]:
export_dir = r'C:\Users\1\Downloads\export_dir'
dataset_type = fo.types.TFObjectDetectionDataset
samples.export(
    export_dir=export_dir, dataset_type=dataset_type)

Directory 'C:\Users\1\Downloads\export_dir' already exists; export will be merged with existing files
 100% |█████████| 13/13 [244.8ms elapsed, 0s remaining, 57.2 samples/s]      
